In [6]:
import sympy
from sympy import Derivative, Function, Symbol, symbols, Eq, pi, cos, sin, exp, log, oo
from sympy import Function, dsolve, Derivative, checkodesol
from sympy import fps, Rational
from sympy import pprint, Matrix, eye, zeros
from sympy import Inverse
from utils import *
z = Symbol('z')

Restate the problem that we want to solve: 

$$z^k \frac{d\psi}{dz} = A(z)\psi$$, where $\psi$ is a vector-valued function and $A(z)$ as a given matrix-valued function is holomorphic.

In the general case where we can't find it directly, we use Formal Gauge transform to simplify the system, and solve it in diagonal version. Then, $\hat{\Phi}(u, z) = \hat{\Psi}(t(u, z)) \hat{\Psi}^{-1}(s(u,z))$ is the formal universal solution on Sto_k, while since $\Phi$ is unique, then this means that $\hat{\Phi}$ is the power series of $\Phi$, so it must be convergent since $\Phi$ is entire.

To do the Gauge transformation, the core idea is to transfer it to the case where $z^k \frac{d\phi}{dz} = (D_0+zD_1+...z^{k-1}D_{k-1})\phi$, where $D_i$ are diagonal, which can be easily solve by entries on both sides.

In order to solve this problem, we first need to make a change of variables $\psi' = F \psi$, whereas $F = 1+zF_1 + z^2F_2 + ...$


In this step, we let $F = (I+zH_1)(I+z^2H_2)...$

where $H_p = \begin{cases} {ad}^{-1}_{A_0}(A_p^{OD}) \text{   if   } k > 1 \\ ({ad}_{A_0} - p)^{-1}(A_p^{OD}) \text{    if   } k = 1  \end{cases}$

After this transform, the ODE becomes $z^k \frac{d\psi'}{dz} = (diagonal(A_0) + diagonal(A_1)z + ...) \psi'$

After doing this, we make the second Gauge Transform to transform it to the finite case, 

which is the transform $K = \exp(-\int{D_k+D_{k+1}z+...})$

Finally, the simplified system is $z^k \frac{d\psi''}{dz} = (diagonal(A_0) + diagonal(A_1)z + ...+ diagonal(A_{k-1})z^{k-1}) \psi''$

and the overall Gauge transform is $(KF)$[...], and the inverse is $(KF)^{-1}$[...], so $\psi = (KF)^{-1}[\psi'']$

After we get the fundamental solution $\Psi$, we can construct $\Phi$ on the Sto_k in a similar way.

Example1: 

$z^2 \psi' = \begin{pmatrix} \cos(z)+1 & \sin(z) \\ -\sin(z) & \cos(z) \end{pmatrix} \psi$ 

We are working on Sto_2

We trancate the Gauge Transform series up to some given order 

In [17]:
A = Matrix([[cos(z)+1, sin(z)], [-sin(z), cos(z)]])
n = 2
k = 2
order = 5

In [19]:
K_trunc, F_trunc, Total = get_Gauge_up_to_order(A, k, order)

In [20]:
K_trunc

Matrix([
[exp(z**2*(-3*z**2 + 80*z + 120)/1440)/2 + exp(z**2*(3*z**2 + 80*z - 120)/1440)/2, exp(z**2*(-3*z**2 + 80*z + 120)/1440)/2 - exp(z**2*(3*z**2 + 80*z - 120)/1440)/2],
[exp(z**2*(-3*z**2 + 80*z + 120)/1440)/2 - exp(z**2*(3*z**2 + 80*z - 120)/1440)/2, exp(z**2*(-3*z**2 + 80*z + 120)/1440)/2 + exp(z**2*(3*z**2 + 80*z - 120)/1440)/2]])

In [21]:
F_trunc

Matrix([
[ 1 - z**4/6, -z**3/6 + z],
[-z**3/6 + z,  1 - z**4/6]])

In [22]:
Total

Matrix([
[-z*(z**2 - 6)*(exp(z**2*(-3*z**2 + 80*z + 120)/1440) - exp(z**2*(3*z**2 + 80*z - 120)/1440))/12 - (z**4 - 6)*(exp(z**2*(-3*z**2 + 80*z + 120)/1440) + exp(z**2*(3*z**2 + 80*z - 120)/1440))/12, -z*(z**2 - 6)*(exp(z**2*(-3*z**2 + 80*z + 120)/1440) + exp(z**2*(3*z**2 + 80*z - 120)/1440))/12 - (z**4 - 6)*(exp(z**2*(-3*z**2 + 80*z + 120)/1440) - exp(z**2*(3*z**2 + 80*z - 120)/1440))/12],
[-z*(z**2 - 6)*(exp(z**2*(-3*z**2 + 80*z + 120)/1440) + exp(z**2*(3*z**2 + 80*z - 120)/1440))/12 - (z**4 - 6)*(exp(z**2*(-3*z**2 + 80*z + 120)/1440) - exp(z**2*(3*z**2 + 80*z - 120)/1440))/12, -z*(z**2 - 6)*(exp(z**2*(-3*z**2 + 80*z + 120)/1440) - exp(z**2*(3*z**2 + 80*z - 120)/1440))/12 - (z**4 - 6)*(exp(z**2*(-3*z**2 + 80*z + 120)/1440) + exp(z**2*(3*z**2 + 80*z - 120)/1440))/12]])

In [23]:
Total_inverse = Total.inverse()

In [24]:
Total_inverse

Matrix([
[(-3*z**8*exp(-z**4/240 + z**3/9 + z**2/6) - 6*z**8*exp(-z**4/480 + z**3/18 + z**2/12)*exp(z**4/480 + z**3/18 - z**2/12) - 3*z**8*exp(z**4/240 + z**3/9 - z**2/6) - 6*z**7*exp(-z**4/240 + z**3/9 + z**2/6) + 6*z**7*exp(z**4/240 + z**3/9 - z**2/6) - 3*z**6*exp(-z**4/240 + z**3/9 + z**2/6) + 6*z**6*exp(-z**4/480 + z**3/18 + z**2/12)*exp(z**4/480 + z**3/18 - z**2/12) - 3*z**6*exp(z**4/240 + z**3/9 - z**2/6) + 36*z**5*exp(-z**4/240 + z**3/9 + z**2/6) - 36*z**5*exp(z**4/240 + z**3/9 - z**2/6) + 72*z**4*exp(-z**4/240 + z**3/9 + z**2/6) + 72*z**4*exp(z**4/240 + z**3/9 - z**2/6) + 36*z**3*exp(-z**4/240 + z**3/9 + z**2/6) - 36*z**3*exp(z**4/240 + z**3/9 - z**2/6) - 108*z**2*exp(-z**4/240 + z**3/9 + z**2/6) + 216*z**2*exp(-z**4/480 + z**3/18 + z**2/12)*exp(z**4/480 + z**3/18 - z**2/12) - 108*z**2*exp(z**4/240 + z**3/9 - z**2/6) - 216*z*exp(-z**4/240 + z**3/9 + z**2/6) + 216*z*exp(z**4/240 + z**3/9 - z**2/6) - 108*exp(-z**4/240 + z**3/9 + z**2/6) - 216*exp(-z**4/480 + z**3/18 + z**2/12)*ex

Note that Gauge Transform is a group action, meaning that $F_1[F_2[A]] = (F_1F_2)[A]$ so the inverse Gauge Transform is the Gauge Transform of the inverse Matrix, and composed Gauge Transform is the transform of the product Matrix.

Finally, the equation is $z^k \psi' = diag(A_0) + diag(A_1)z + ... + diag(A_{k-1})z^{k-1}$

In [37]:
Psi, Phi = solve_phi(k, A)

In [38]:
Psi

Matrix([
[exp(-2/z),         0],
[        0, exp(-1/z)]])

In [39]:
Phi

Matrix([
[exp(2*(1 - exp(-u*z))/z),                      0],
[                       0, exp((1 - exp(-u*z))/z)]])

The final solution is $T^{-1}[\Psi]$ (we use $T$ to denote the total Gauge Transform that we applied to simplify the system)

$T^{-1}$ is the Total_inverse that we just found above.

Example2: 

$z^2 \psi' = \begin{pmatrix} z^4+3z^3+1 & z \\ z^2 & z^3+5z^2+7 \end{pmatrix} \psi$ 

We are working on Sto_2

We trancate the Gauge Transform series up to some given order 

In [43]:
A = Matrix([[z**4+3*z**3+1, z], [z**2, z**3+5*z**2+7]])
n = 2
k = 2
order = 10

In [45]:
K_trunc, F_trunc, Total = get_Gauge_up_to_order(A, k, order)

In [46]:
Total

Matrix([
[                    (36 - z**3)*exp(z**2/72)/36,  -z*exp(z**2/72)/6],
[(z**3 - 2*(1 - exp(z**2/72))*(z**3 - 36))/(6*z), 2*exp(z**2/72) - 1]])

In [47]:
Total_inverse = Total.inverse()

In [48]:
Total_inverse

Matrix([
[              (-exp(z**2/72) + 2*exp(z**2/36))*exp(-z**2/36),         z/6],
[(-2*z**3 + z**3*exp(-z**2/72) + 72 - 72*exp(-z**2/72))/(6*z), 1 - z**3/36]])

Note that Gauge Transform is a group action, meaning that $F_1[F_2[A]] = (F_1F_2)[A]$ so the inverse Gauge Transform is the Gauge Transform of the inverse Matrix, and composed Gauge Transform is the transform of the product Matrix.

Finally, the equation is $z^k \psi' = diag(A_0) + diag(A_1)z + ... + diag(A_{k-1})z^{k-1}$

In [53]:
Psi, Phi = solve_phi(k, A)

In [54]:
Psi

Matrix([
[exp(-1/z),         0],
[        0, exp(-7/z)]])

In [55]:
Phi

Matrix([
[exp((1 - exp(-u*z))/z),                        0],
[                     0, exp(7*(1 - exp(-u*z))/z)]])

The final solution is $T^{-1}[\Psi]$ (we use $T$ to denote the total Gauge Transform that we applied to simplify the system)

$T^{-1}$ is the Total_inverse that we just found above.

In [59]:
T_inverse_Psi = (Total * Psi + Derivative(Total, z).simplify() * z**k) * Total_inverse 
T_inverse_Psi = T_inverse_Psi.simplify()

In [60]:
T_inverse_Psi

Matrix([
[                                                                                                                                            ((z*(z**2 + 36)*exp(7/z) + 36)*(-z**3 + 2*(z**3 - 36)*exp(z**2/72) + 72)*exp((z**3 + 36)/(36*z)) + (exp(z**2/72) - 2*exp(z**2/36))*(z**3*(z**3 + 108*z - 36)*exp(1/z) + 36*z**3 - 1296)*exp((z**3 + 504)/(72*z)))*exp(z**2/72 - (z**3 + 192)/(24*z))/1296,                                                                                                                                                                         z*(-2*z**4*exp(7/z) - z**3*exp(6/z) + z**3 - 36*z*exp(7/z) + 36*exp(6/z) - 36)*exp(z**2/72 - 7/z)/216],
[-((exp(z**2/72) - 2*exp(z**2/36))*(18*z**3 - z*(18*z**3 - z**2*(108*z*(exp(z**2/72) - 1) + 54*z + (z**3 - 36)*exp(z**2/72)) + 36*(z**3 - 36)*(exp(z**2/72) - 1))*exp(1/z) + 36*(z**3 - 36)*(exp(z**2/72) - 1))*exp((z**3 + 504)/(72*z)) + (-z**3 + 2*(z**3 - 36)*exp(z**2/72) + 72)*(z**3*exp((z**3 + 504)/(72*z)) + 36*exp(z**2/72) -

In [ ]:
Final_Phi = get_phi_from_psi(T_inverse_Psi, k)
## This will take much long time to run.

In [ ]:
Final_Phi

Even though we could Finally get $\Phi$ of the original system, the last step will taks too much time to compute, which is a limitation that we could optimize, yet nevertheless the computational ability has already surpassed human being.